In [1]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyvista
import ipywidgets
import ipyevents
import pyvistaqt
import os

import mne
from mne import concatenate_raws

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_curve, auc
from sklearn.utils import class_weight

import tensorflow as tf
from tensorflow.keras import layers, models

import scipy.signal as signal
from scipy.signal import hilbert

In [2]:
%matplotlib qt

### Load the participant data from NREM 2 and 3

In [3]:
participant_large_ids = ["013", "014", "015", "016", "017", "018", "019", "020", "021", "022", "023", "024", "025", "026", "027", "028", "029", "030", "031", "032", "033", "034", "035", "036",
               "037", "038", "039", "040", "041", "042", "043", "044", "045", "046", "047", "048",
               "049", "050", "051", "052", "053", "054", "055", "056", "057", "058", "059", "060",
               "061", "062", "063", "064", "065", "066", "067", "068", "069", "070", "071", "072",
               "073", "074", "075", "076", "077", "078", "079", "080", "081", "082", "083", "084"]

combined_raw_large_data = {}
# will store the raw objects in a dictionary
base_path = r"C:\EEG DATA"

# for loop through the participants to load the data
for pid in participant_large_ids:
    file_path = fr"{base_path}\{pid}\concatenated\{pid}_concatenated_raw.fif"
    combined_raw_large_data[pid] = mne.io.read_raw_fif(file_path, preload=True)

# if want to access participant 067
# raw_067 = combined_raw_data["067"]

Opening raw data file C:\EEG DATA\020\concatenated\020_concatenated_raw.fif...
Isotrak not found
    Range : 1470000 ... 7845026 =   2940.000 ... 15690.052 secs
Ready.
Reading 0 ... 6375026  =      0.000 ... 12750.052 secs...
Opening raw data file C:\EEG DATA\033\concatenated\033_concatenated_raw.fif...
Isotrak not found
    Range : 90000 ... 7560030 =    180.000 ... 15120.060 secs
Ready.
Reading 0 ... 7470030  =      0.000 ... 14940.060 secs...
Opening raw data file C:\EEG DATA\046\concatenated\046_concatenated_raw.fif...
Isotrak not found
    Range : 300000 ... 8280014 =    600.000 ... 16560.028 secs
Ready.
Reading 0 ... 7980014  =      0.000 ... 15960.028 secs...
Opening raw data file C:\EEG DATA\067\concatenated\067_concatenated_raw.fif...
Isotrak not found
    Range : 825000 ... 9180031 =   1650.000 ... 18360.062 secs
Ready.
Reading 0 ... 8355031  =      0.000 ... 16710.062 secs...
Opening raw data file C:\EEG DATA\081\concatenated\081_concatenated_raw.fif...
Isotrak not found
   

### Set 6

In [4]:
train_6_large_ids = ["013", "014", "015", "016", "017", "018", "019", "020", "021", "022", "023", "024",
               "025", "026", "027", "028", "029", "030", "031", "032", "033", "034", "035", "036",
               "037", "038", "039", "040", "041", "042", "043", "044", "045", "046", "047", "048",
               "049", "050", "051", "052", "053", "054", "055", "056", "057", "058", "059", "060",
               "061", "062", "063", "064", "065", "066", "067", "068", "069", "070", "071", "072"]
test_6_large_ids = ["073", "074", "075", "076", "077", "078", "079", "080", "081", "082", "083", "084"]

# concatenation
train_6_large_raw = mne.concatenate_raws([combined_raw_large_data[pid] for pid in train_6_large_ids])
test_6_large_raw = mne.concatenate_raws([combined_raw_large_data[pid] for pid in test_6_large_ids])

print(train_6_large_raw.times[-1])
print(test_6_large_raw.times[-1])

65220.19
12750.052


In [5]:
# directory for saving
save_dir = os.path.join(base_path, "combined_sets")
os.makedirs(save_dir, exist_ok=True)

# Train set
train_6_large_save_path = os.path.join(save_dir, "train_6_large_raw.fif")
train_6_large_raw.save(train_6_large_save_path, overwrite=True)
print(f"Train set saved to {train_6_large_save_path}")

# Test set
test_6_large_save_path = os.path.join(save_dir, "test_6_large_raw.fif")
test_6_large_raw.save(test_6_large_save_path, overwrite=True)
print(f"Test set saved to {test_6_large_save_path}")

Overwriting existing file.
Writing C:\EEG DATA\combined_sets\train_1_raw.fif
Closing C:\EEG DATA\combined_sets\train_1_raw.fif
[done]
Train set saved to C:\EEG DATA\combined_sets\train_1_raw.fif
Overwriting existing file.
Writing C:\EEG DATA\combined_sets\test_1_raw.fif
Closing C:\EEG DATA\combined_sets\test_1_raw.fif
[done]
Test set saved to C:\EEG DATA\combined_sets\test_1_raw.fif
